# Molecular Quantum Mechanics (CB2070)

## SOLUTION Computer lab 2: Fock matrices and the SCF procedure
---
Name: J.H. Andersen with stolen content from VT24 and VT25 student submissions

Date: July 2025

---

In [ ]:
import veloxchem as vlx

import numpy as np
import matplotlib.pyplot as plt

# format the numpy array printout
np.set_printoptions(precision=8, suppress=True, linewidth=132)

In [ ]:
mol_str = """
    H    0.0000    0.7604    0.7295
    H    0.0000   -0.7604    0.7295
    O    0.0000    0.0000    0.1410
"""

molecule = vlx.Molecule.read_str(mol_str, units='angstrom')
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz")

n = basis.get_dimensions_of_basis()
print('Number of contracted basis functions:', n)

nocc = molecule.number_of_electrons() // 2
print('Number of occupied MOs:', nocc)

### 1. SCF optimization

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_results = scf_drv.compute(molecule, basis)

# list the keys in the SCF result dictionary
print('Keys in the SCF result dictionary:')
for key in scf_results.keys():
    print('\t' + key)

# obtain tensors
E = scf_results['E_alpha']
C = scf_results['C_alpha']
D = scf_results['D_alpha']
F = scf_results['F_alpha']
S = scf_results['S']

print('\nC, D, F, S matrix shapes:\n', np.shape(C), np.shape(D), np.shape(F), np.shape(S))

### 2. Diagonalize the overlap matrix, S

Complete the formula
$$
\mathbf{S}_\mathrm{diag} = \mathbf{U}^\dagger \mathbf{S} \mathbf{U}
$$

In [ ]:
# Implement the formula
sigma, U = np.linalg.eigh(S)

In [ ]:
# Check that all eigenvalues are positive
print(sigma > 0.0)

In [ ]:
# Verify that U*U = I

# this can be done with matmul
UdagU = np.matmul(U.T, U)
print(np.diag(UdagU))

# ...or einsum
#UdagU = np.einsum("ki,kj->ij", U, U)
#print(np.diag(UdagU))

print("Matrices are equal to a 1.0e-6 threshold:", np.allclose(UdagU, np.identity(n), atol=1.0e-6))

### 3. The non-unitary Hermitian transformation matrix
Complete the formula

$$
\mathbf{X} =  \mathbf{S^{-1/2}} = \mathbf{U} \mathbf{S}_\mathrm{diag}^{-1/2} \mathbf{U}^{\dagger}
$$

This is valid from following argument:

Let $\mathbf{A}$ be a matrix with the decomposition $\mathbf{U}^{\dagger} A \mathbf{U}  = \mathbf{A}_\mathrm{diag} \implies A = \mathbf{U} \mathbf{A}_\mathrm{diag} \mathbf{U}^{\dagger} $. Then a function of $A$ can be written as,

$$
f(\mathbf{A}) = \sum_{n=0}^{\infty} a_n (\mathbf{A})^n = \sum_{n=0}^{\infty} a_n (\mathbf{U} \mathbf{A}_\mathrm{diag} \mathbf{U}^{\dagger})^n 
$$

For $n=2$, the matrix product is expanded, $(\mathbf{U} \mathbf{A}_\mathrm{diag} \mathbf{U}^{\dagger})^2 = \mathbf{U} \mathbf{A}_\mathrm{diag} \mathbf{U}^{\dagger} \mathbf{U} \mathbf{A}_\mathrm{diag} \mathbf{U}^{\dagger} = \mathbf{U} \mathbf{A}_\mathrm{diag}^2 \mathbf{U}^{\dagger}$, since $\mathbf{U}^{\dagger} \mathbf{U} = 1$. This holds for any $n$, and thus

$$
f(\mathbf{A}) = \sum_{n=0}^{\infty} a_n  \mathbf{U} (\mathbf{A}_\mathrm{diag})^n \mathbf{U}^{\dagger} = \mathbf{U} \left[ \sum_{n=0}^{\infty} a_n (\mathbf{A}_\mathrm{diag})^n \right] \mathbf{U}^{\dagger} = \mathbf{U} f(\mathbf{A}_\mathrm{diag} ) \mathbf{U}^{\dagger}
$$



In [ ]:
# Implement the formula

# determine X from the unitary transformation matrix and eigenvalues of S

# with matmul
X = np.matmul(U, np.matmul(np.diag(1.0 / np.sqrt(sigma)) , U.T) )

# ...or with einsum
#X = np.einsum("ik,k,jk->ij", U, 1 / np.sqrt(sigma), U)


### 4. The transformed Fock matrix

We define a new set of basis functions

$$
| \tilde{\chi}_\alpha \rangle = \sum_\beta |\chi_\beta \rangle X_{\beta\alpha};\quad
|\tilde{\boldsymbol{\chi}} \rangle = |\boldsymbol{\chi} \rangle \mathbf{X}
$$

#### 4.a Derive the relation between MO coefficients in the original and new basis sets.

MO coefficients in the original basis set are collected in the matrix $\mathbf{C}$,

$$
%| \phi_a \rangle = \sum_\alpha  C_{\alpha a} |\chi_{\alpha} \rangle;\quad
|{\boldsymbol{\phi}} \rangle = |\boldsymbol{\chi} \rangle \mathbf{C}
$$

Multiplying by the inverse of $\mathbf{X}$ from the right in the equation defining the new basis:

$$
|\tilde{\boldsymbol{\chi}} \rangle \mathbf{X}^{-1} = |\boldsymbol{\chi} \rangle \mathbf{X} \mathbf{X}^{-1} = |\boldsymbol{\chi} \rangle
$$

We can then write

$$
|{\boldsymbol{\phi}} \rangle = |\tilde{\boldsymbol{\chi}} \rangle \mathbf{X}^{-1} \mathbf{C}
$$

which defines the MO coefficients for the new set of basis functions:

$$
\mathbf{C'} = \mathbf{X}^{-1} \mathbf{C}
$$

#### 4.b Derive the transformation of the Fock matrix.
In the original basis, the Fock matrix reads

$$
\mathbf{F} \mathbf{C} = \mathbf{S} \mathbf{C} \boldsymbol{\varepsilon}
$$

Substituting $\mathbf{C} = \mathbf{X} \mathbf{C'}$ in the Fock equation

$$
\mathbf{F} \mathbf{X} \mathbf{C'} = \mathbf{S} \mathbf{X} \mathbf{C'} \boldsymbol{\varepsilon}
$$

and multiplying from the left by $\mathbf{X}^{\dagger}$

$$
\mathbf{X}^{\dagger} \mathbf{F} \mathbf{X} \mathbf{C'} = \mathbf{X}^{\dagger} \mathbf{S} \mathbf{X} \mathbf{C'} \boldsymbol{\varepsilon}
$$

If the Fock-equation should be transformed into an eigenvalue equation, $\mathbf{X}^{\dagger} \mathbf{S} \mathbf{X}$ must be equal to the identity matrix in agreement with $\mathbf{X} = \mathbf{S}^{-1/2}$,

$$
(\mathbf{S^{-1/2}})^{\dagger} \mathbf{S} \mathbf{S^{-1/2}} = \mathbf{S^{-1/2}} \mathbf{S} \mathbf{S^{-1/2}} = \mathbb{1}
$$

where the first equality is due to the overlap matrix being Hermitian. Thus, the transformed Fock equation reads

$$
\mathbf{F}' \mathbf{C'} = \mathbf{C'} \boldsymbol{\varepsilon}
$$

where the transformed Fock matrix is defined as,

$$
\mathbf{F}' = \mathbf{X}^{\dagger} \mathbf{F} \mathbf{X}
$$

In [ ]:
# Implement the transformation of the Fock matrix

# with matmul
F_prime = np.matmul(X.T, np.matmul(F, X))

# ...or with einsum
#F_prime = np.einsum("ik,kl,lj->ij", X, F, X)

### 5. Solve the transformed HF equation

In [ ]:
# Implement the diagonalization of the transformed Fock matrix
epsilon, C_prime = np.linalg.eigh(F_prime)

### 6. Determine the MO coefficients in the original basis from the transformed MO coefficients

In [ ]:
# Implement the transformation of C'

# with matmul
C_orig = np.matmul(X, C_prime)

# ...or with einsum
#C_orig = np.einsum('ki, kj -> ij', X, C_prime)

### 7. Determine the density matrix

Complete the formula

$$
D_{\alpha\beta} = \sum_{i=1}^N = C_{\alpha i}^* C_{\beta i} 
$$

where $N$ is the number of electrons in the system. Since we are treating only the $\alpha$ spins, in practice we sum over $N/2$.

In [ ]:
# Implement the calculation of the density matrix

# with matmul
D_new = np.matmul(C_orig[:, :nocc], C_orig[:, :nocc].T)

# ...or with einsum
#D_new = np.einsum('ai, bi -> ab', C_orig[:,:nocc], C_orig[:,:nocc])


#### Compare your density matrix with that from the previous SCF calculation. Has the SCF procedure converged?

In [ ]:
# compare the density matrices
print("Matrices are equal to a 1.0e-7 threshold:", np.allclose(D, D_new, atol=1.0e-7))

The density matrices have converged to a threshold of $10^{-7}$. This was expected since the original density matrix is taken from a converged SCF calculation.

## THE END